# 神经网络的训练过程
- 定义神经网络
- 迭代输入数据
- 神经网络计算输出
- 计算损失
- 反向传递梯度回到网络的参数
- 更新网络的权重

![AlexNet以及中间层的输出效果](images/AlexNet以及中间层的输出效果.png)

# 定义神经网络和训练流程

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self): # 定义神经网络结构，输入数据 1*32*32
        super(Net, self).__init__()
        # 第一层（卷积层）
        self.conv1 = nn.Conv2d(1, 6, 3) # 输入频道1， 输出频道6， 卷积3*3
        # 第二层（卷积层）
        self.conv2 = nn.Conv2d(6, 16, 3) # 输入频道6， 输出频道16， 卷积3*3
        # 第三层（全连接层）
        self.fc1 = nn.Linear(16*28*28, 512) # 输入维度16*28*28=12544，输出维度512.这里的28，是因为经过每一层都会减2，32-2-2=28
        # 第四层（全连接层）
        self.fc2 = nn.Linear(512, 64) # 输入维度512， 输出维度64
        # 第五层（全连接层）
        self.fc3 = nn.Linear(64, 2) # 输入维度64， 输出维度2
        
    def forward(self, x): # 定义数据流向
        x = self.conv1(x)
        x = F.relu(x) # 激活函数
        
        x = self.conv2(x)
        x = F.relu(x)
        
        x = x.view(-1, 16*28*28) # 将卷积层的输出张量重新调整形状，以便输入到全连接层，这个操作通过被成为“展平”。-1表示让torch自动计算该维度的大小，以确保元素总数不变，16 * 28 * 28：这是卷积层输出的特征图的总元素数量。16 是通道数，28 是高度，28 是宽度
        x = self.fc1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        
        x = self.fc3(x) # 在神经网络的最后一层通常不需要使用激活函数，最后一层的输出通常会直接传递给损失函数进行计算
        
        return x

In [ ]:
net = Net()
print(net)

# 运行神经网络与计算损失

In [ ]:
# 生成随机输入
input_data = torch.randn(1, 1, 32, 32) # 生成随机输入，批量大小为 1，通道数为 1，高度和宽度为 32
print(input_data)
print(input_data.size()) # 返回张量的形状，功能同input_data.shape

In [ ]:
# 运行神经网络
out = net(input_data)
print(out)
print(out.size())

In [ ]:
# 随机生成真实值
target = torch.randn(2)
target = target.view(1, -1) # 将张量 target 重新调整形状（reshape）为一个二维张量，其中第一维的大小为 1，第二维的大小由 PyTorch 自动计算以确保元素总数不变
print(target)

In [ ]:
criterion = nn.L1Loss() # 定义损失函数
loss = criterion(out, target) # 计算损失
print(loss)

# 反向传递与权值更新

In [ ]:
# 反向传递
net.zero_grad() # 清零梯度
loss.backward() # 自动计算梯度、反向传递

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)
optimizer.step() # 根据计算的梯度（gradients）和优化器的配置（如学习率`lr`）来更新模型的参数，以最小化损失函数。

In [ ]:
out = net(input_data)
print(out)
print(out.size())

In [ ]:
loss = criterion(out, target) # 计算损失
print(loss)

# 神经网络中损失函数的用途
- 损失函数代表了预测值与真实值的差
- 常见损失函数
    - mean absolute loss (L1)
    - mean squared loss (MSE)
    - cross entropy loss
    - KL-divergence
- 损失函数的用途
    - 用于计算梯度、反向传递、更新网络权重
    - 判断模型是否训练好了
        - 判断过度拟合、欠拟合

![正确率与训练时间](images/正确率与epoch.png)